In [1]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Load the pre-trained DenseNet121 model
model = models.densenet121(pretrained=True)
model.eval()

# Step 2: Load the images
input_directory = "./output_images_folder(DenseNet)"
output_original_directory = "./original_incorrect_predictions_folder(PGD)_DenseNet"
output_adversarial_directory = "./adversarial_images_folder(PGD)_DenseNet"

input_images = [os.path.join(input_directory, filename) for filename in os.listdir(input_directory) if filename.endswith(".JPEG")]

# Step 3: Define the PGD attack function
def pgd_attack(model, image, target, epsilon, alpha, num_iter):
    perturbed_image = image.clone().detach().requires_grad_(True)
    for i in range(num_iter):
        output = model(perturbed_image)
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        with torch.no_grad():
            perturbed_image += alpha * perturbed_image.grad.sign()
            perturbed_image = torch.min(torch.max(perturbed_image, image - epsilon), image + epsilon)
            perturbed_image = torch.clamp(perturbed_image, 0, 1)
            perturbed_image.requires_grad_(True)
    return perturbed_image

# Step 4: Generate adversarial examples with PGD attack
epsilon = 0.01  # Adjust this value as needed
alpha = 0.01    # Adjust this value as needed
num_iter = 20   # Adjust this value as needed

successful_adversarial_examples = []
corres_org_img=[]

for input_image in input_images:
    # Load the original image
    original_image = Image.open(input_image).convert("RGB")
    
    # Convert the PIL image to a PyTorch tensor and apply the necessary preprocessing
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
    image_tensor = preprocess(original_image).unsqueeze(0)
    image_tensor.requires_grad = True

    # Forward pass to get the predictions
    output = model(image_tensor)
    original_label = torch.argmax(output, 1).item()

    # Generate the target label for the PGD attack (choose any label other than the original label)
    target_label = torch.randint_like(output, 0, output.shape[1])

    # Generate the perturbed image using the PGD attack
    perturbed_image = pgd_attack(model, image_tensor, target_label, epsilon, alpha, num_iter)

    # Forward pass with the perturbed image
    perturbed_output = model(perturbed_image)

    # Check if the perturbed image's predicted label differs from the original label
    perturbed_label = torch.argmax(perturbed_output, 1).item()
    if perturbed_label != original_label:
        # Save the perturbed image as a successful adversarial example
        successful_adversarial_examples.append(perturbed_image)
        corres_org_img.append(input_image)
        # Save the original image to output_original_directory
        os.makedirs(output_original_directory, exist_ok=True)
        original_image_name = os.path.basename(input_image)
        output_original_image_path = os.path.join(output_original_directory, original_image_name)
        original_image.save(output_original_image_path)
        
        # Save the perturbed image to output_adversarial_directory
        os.makedirs(output_adversarial_directory, exist_ok=True)
        perturbed_image_name = f"perturbed_{original_image_name}"
        output_perturbed_image_path = os.path.join(output_adversarial_directory, perturbed_image_name)
        perturbed_image_pil = transforms.ToPILImage()(perturbed_image.squeeze(0))
        perturbed_image_pil.save(output_perturbed_image_path)




C:\Users\DandaC8719\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\DandaC8719\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
# Print the length of successful adversarial examples
print("Number of successful adversarial examples:", len(successful_adversarial_examples))
print(len(corres_org_img))

Number of successful adversarial examples: 3566
3566


In [2]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import json

# Step 1: Load the pre-trained DenseNet121 model
model = models.densenet121(pretrained=True)
model.eval()

# Load ImageNet class labels
with open("C:/Users/DandaC8719/Documents/capstone_overall_code/imagenet_class_index.json", "r") as f:
    imagenet_classes = json.load(f)

# Convert class labels to dictionary format
imagenet_classes = {int(k): v for k, v in imagenet_classes.items()}

# Step 3: Define the PGD attack function
def pgd_attack(model, image, target, epsilon, alpha, num_iter):
    perturbed_image = image.clone().detach().requires_grad_(True)
    for i in range(num_iter):
        output = model(perturbed_image)
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        with torch.no_grad():
            perturbed_image += alpha * perturbed_image.grad.sign()
            perturbed_image = torch.min(torch.max(perturbed_image, image - epsilon), image + epsilon)
            perturbed_image = torch.clamp(perturbed_image, 0, 1)
            perturbed_image.requires_grad_(True)
    return perturbed_image

# Step 4: Generate adversarial examples with PGD attack
epsilon = 0.01  # Adjust this value as needed
alpha = 0.01    # Adjust this value as needed
num_iter = 20   # Adjust this value as needed

def get_labels(image_path):
    from PIL import Image  # Import Image module here
    
    # Load the original image
    original_image = Image.open(image_path).convert("RGB")
    
    # Convert the PIL image to a PyTorch tensor and apply the necessary preprocessing
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image_tensor = preprocess(original_image).unsqueeze(0)
    image_tensor.requires_grad = True

    # Forward pass to get the predictions
    output = model(image_tensor)
    original_label_id = torch.argmax(output, 1).item()

    # Calculate the confidence percentage for the original prediction
    original_confidence = torch.softmax(output, dim=1)[0, original_label_id].item() * 100

    # Get class names for original label
    original_label_name = imagenet_classes[original_label_id]

    # Calculate the loss
    loss = nn.CrossEntropyLoss()(output, torch.tensor([original_label_id]))

    # Zero all existing gradients
    model.zero_grad()

    # Backpropagate the loss to compute gradients
    loss.backward()

    # Collect the gradients of the input image
    data_grad = image_tensor.grad.data

    # Generate the target label
    target = torch.tensor([original_label_id])

    # Generate the perturbed image using the PGD attack
    perturbed_image = pgd_attack(model, image_tensor, target, epsilon, alpha, num_iter)

    # Forward pass with the perturbed image
    perturbed_output = model(perturbed_image)

    # Get the adversarial label
    perturbed_label_id = torch.argmax(perturbed_output, 1).item()

    # Calculate the confidence percentage for the adversarial prediction
    perturbed_confidence = torch.softmax(perturbed_output, dim=1)[0, perturbed_label_id].item() * 100

    # Get class names for adversarial label
    perturbed_label_name = imagenet_classes[perturbed_label_id]
    
    return (original_label_id, original_label_name, original_confidence), (perturbed_label_id, perturbed_label_name, perturbed_confidence)


image_path = "./original_incorrect_predictions_folder(PGD)_DenseNet/ILSVRC2012_val_00024749.JPEG"  
(original_label_id, original_label_name, original_confidence), (adversarial_label_id, adversarial_label_name, adversarial_confidence) = get_labels(image_path)
print("Original Label ID:", original_label_id)
print("Original Label Name:", original_label_name)
print("Original Confidence:", original_confidence)
print("Adversarial Label ID:", adversarial_label_id)
print("Adversarial Label Name:", adversarial_label_name)
print("Adversarial Confidence:", adversarial_confidence)

Original Label ID: 148
Original Label Name: ['n02071294', 'killer_whale']
Original Confidence: 70.89036703109741
Adversarial Label ID: 693
Adversarial Label Name: ['n03873416', 'paddle']
Adversarial Confidence: 70.52967548370361
